# Тема “POS-tagger и NER”

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы
 
Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/

проверить NER из nltk/spacy/deeppavlov

написать свой нер попробовать разные подходы

передаём в сетку токен и его соседей

передаём в сетку только токен

свой вариант

сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [4]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.8 MB/s 


In [5]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install ipymarkup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=eaa45dae3517b19f1da28dd0a2bd11ba82f2b7d80358d35f0107ee5377a57e9e
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [8]:
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-07-12 09:06:24.575979: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/usr/local/lib/python3.7/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.4.0                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        


In [9]:
!pip install slovnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 34.0 MB/s 


In [51]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 100 kB/s 
     |████████████████████████████████| 41 kB 123 kB/s 


In [22]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report


import os
import corus
import spacy
from spacy import displacy
import ru_core_news_sm
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

In [13]:
full_train = pyconll.load_from_file('/ru_syntagrus-ud-train-a.conllu.txt')
full_test = pyconll.load_from_file('/ru_syntagrus-ud-dev.conllu.txt')

In [14]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [15]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [16]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [17]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_U = unigram_tagger.evaluate(fdata_test)
display(accuracy_U)

0.823732013802982

In [18]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_B = bigram_tagger.evaluate(fdata_test)
display(accuracy_B)

0.6093886320724006

In [19]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_T = trigram_tagger.evaluate(fdata_test)
display(accuracy_T)

0.1778631421316492

In [20]:
list_1 = [UnigramTagger, BigramTagger]

list_2 = [UnigramTagger, TrigramTagger]

list_3 = [BigramTagger, TrigramTagger]

list_4 = [UnigramTagger, BigramTagger, TrigramTagger]

accuracy_N = []

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

for list_N in [list_1, list_2, list_3, list_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         list_N,  
                         backoff = backoff) 

    accuracy_N.append(tag.evaluate(fdata_test))

accuracy_N

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [23]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 'UnigramTagger+BigramTagger+TrigramTagger'], 'Accuracy' : [accuracy_U, accuracy_B, accuracy_T] + accuracy_N})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


# Самописный триггер

In [24]:
#собираем токены и лэйблы в списки
train_tok = [] #список токенов
train_label = [] #список теггов
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [25]:
#кодируем целевые метки значениями от 0 до (кол-во классов - 1)
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [27]:
#строим модели логистической регрессии с различными векторайзерами

vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    accuracy_scores.append(acc)

    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [28]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921158
9,"HashingVectorizer(analyzer='char', n_features=...",0.903641
8,"HashingVectorizer(analyzer='char', n_features=...",0.901222
7,"HashingVectorizer(analyzer='char', n_features=...",0.897196
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882224
4,"TfidfVectorizer(ngram_range=(1, 3))",0.666330
3,"CountVectorizer(ngram_range=(1, 3))",0.649712
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.620867


In [29]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923583
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905000
7,"HashingVectorizer(analyzer='char', n_features=...",0.900835
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885165
4,"TfidfVectorizer(ngram_range=(1, 3))",0.656643
3,"CountVectorizer(ngram_range=(1, 3))",0.641139
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.630722


In [30]:
result.sort_values('Accuracy', ascending=False).head(1)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905


In [31]:
result_model.sort_values('f1_score', ascending=False).head(1)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826


In [32]:
result_model_acc.sort_values('Accuracy', ascending=False).head(1)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504


# Задание 2

In [33]:
#просматриваем содержание коллекции
print(os.listdir("/content/drive/MyDrive/Collection3"))

['1006.ann', '1000.txt', '09_01_13i.txt', '09_01_13e.txt', '09_01_13h.txt', '100.txt', '1003.ann', '1004.ann', '1005.ann', '1005.txt', '1002.txt', '1003.txt', '1001.txt', '1002.ann', '1004.txt', '030.txt', '022.txt', '003.txt', '062.txt', '069.txt', '009.txt', '012.txt', '03_12_12g.txt', '04_12_12h_corr.txt', '043.txt', '040.txt', '080.txt', '044.txt', '088.txt', '060.txt', '094.txt', '09_01_13d.txt', '03_12_12a.txt', '036.txt', '098.txt', '09_01_13a.txt', '075.txt', '04_02_13a_abdulatipov.txt', '041.txt', '073.txt', '078.txt', '084.txt', '079.txt', '057.txt', '093.txt', '001.txt', '03_12_12d.txt', '028.txt', '046.txt', '091.txt', '04_12_12f.txt', '071.txt', '068.txt', '038.txt', '034.txt', '04_12_12d.txt', '027.txt', '026.txt', '070.txt', '095.txt', '056.txt', '09_01_13c.txt', '005.txt', '077.txt', '092.txt', '006.txt', '064.txt', '067.txt', '049.txt', '054.txt', '082.txt', '050.txt', '090.txt', '004.txt', '03_12_12h.txt', '010.txt', '03_12_12c.txt', '047.txt', '058.txt', '04_03_13a_s

In [34]:
#собираем только текстовые файлы коллекции
fileDir = r"/content/drive/MyDrive/Collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_txt)

['1000.txt', '09_01_13i.txt', '09_01_13e.txt', '09_01_13h.txt', '100.txt', '1005.txt', '1002.txt', '1003.txt', '1001.txt', '1004.txt', '030.txt', '022.txt', '003.txt', '062.txt', '069.txt', '009.txt', '012.txt', '03_12_12g.txt', '04_12_12h_corr.txt', '043.txt', '040.txt', '080.txt', '044.txt', '088.txt', '060.txt', '094.txt', '09_01_13d.txt', '03_12_12a.txt', '036.txt', '098.txt', '09_01_13a.txt', '075.txt', '04_02_13a_abdulatipov.txt', '041.txt', '073.txt', '078.txt', '084.txt', '079.txt', '057.txt', '093.txt', '001.txt', '03_12_12d.txt', '028.txt', '046.txt', '091.txt', '04_12_12f.txt', '071.txt', '068.txt', '038.txt', '034.txt', '04_12_12d.txt', '027.txt', '026.txt', '070.txt', '095.txt', '056.txt', '09_01_13c.txt', '005.txt', '077.txt', '092.txt', '006.txt', '064.txt', '067.txt', '049.txt', '054.txt', '082.txt', '050.txt', '090.txt', '004.txt', '03_12_12h.txt', '010.txt', '03_12_12c.txt', '047.txt', '058.txt', '04_03_13a_sorokin.txt', '03_12_12b.txt', '059.txt', '051.txt', '033.txt

In [35]:
#заносим данные файлов txt в датасет
text_list = []
for file in documents_txt:
    doc = open('/content/drive/MyDrive/Collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,"""Яблоко"" выдвинуло Геннадия Гудкова в губернат..."
1,\nЯнукович назначил Якименко главой СБУ\nФото ...
2,"""Ювентус"" обыграл ""Милан"" в Кубке Италии\n""Юве..."
3,Политика\nПрезидент назначил Андрея Николаенко...
4,Главой КС Украины стал выходец из Донецка\nПре...
...,...
995,"Стадион ""Зенита"" стоил должности главному стро..."
996,"Анатолий Сердюков был ""эффективным министром о..."
997,"Мать рядового Владислава Челаха, обвиняемого в..."
998,"МОСКВА, 7 ноября - РИА Новости. Назначение Сер..."


# NER из NLTK

In [36]:
#требуется для токенизации
nltk.download('punkt')
#требуется для parts of speech tagging
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [37]:
#пример текста
document = data_text.text[0]

#разбиваем документ на токены и применяем pos tagging (на выходе список кортежей (токен, часть речи))
nltk.pos_tag(nltk.word_tokenize(document))

[('``', '``'),
 ('Яблоко', 'NN'),
 ("''", "''"),
 ('выдвинуло', 'CC'),
 ('Геннадия', 'NNP'),
 ('Гудкова', 'NNP'),
 ('в', 'NNP'),
 ('губернаторы', 'NNP'),
 ('Подмосковья', 'NNP'),
 ('Делегаты', 'NNP'),
 ('конференции', 'NNP'),
 ('выдвигали', 'NNP'),
 ('также', 'NNP'),
 ('кандидатуру', 'NNP'),
 ('``', '``'),
 ('яблочника', 'NN'),
 ("''", "''"),
 ('Кирилла', 'NN'),
 ('Янкова', 'NNP'),
 (',', ','),
 ('однако', 'NNP'),
 ('он', 'NNP'),
 ('снял', 'NNP'),
 ('свою', 'NNP'),
 ('кандидатуру', 'NNP'),
 ('.', '.'),
 ('Таким', 'NN'),
 ('образом', 'NN'),
 (',', ','),
 ('фамилия', 'NNP'),
 ('Гудкова', 'NNP'),
 ('была', 'NNP'),
 ('единственной', 'NNP'),
 ('в', 'NNP'),
 ('бюллетенях', 'NNP'),
 ('для', 'NNP'),
 ('голосования', 'NNP'),
 ('.', '.'),
 ('Конференция', 'VB'),
 ('подмосковного', 'JJ'),
 ('отделение', 'NNP'),
 ('партии', 'NNP'),
 ('``', '``'),
 ('Яблоко', 'NN'),
 ("''", "''"),
 ('в', 'CC'),
 ('субботу', 'JJ'),
 ('выдвинула', 'JJ'),
 ('экс-депутата', 'JJ'),
 ('Госдумы', 'NN'),
 ('Геннадия', 'NNP

In [41]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [43]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [44]:
#распознаем именнованные сущности с помощью классификатора (Person, Organization, GPE)
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Геннадия Гудкова', 'PERSON'),
 ('Госдумы Геннадия Гудкова', 'PERSON'),
 ('Гудков', 'PERSON'),
 ('Гудкова', 'PERSON'),
 ('Кирилла Янкова', 'PERSON'),
 ('Сергей Шойгу', 'PERSON'),
 ('Таким', 'PERSON')}

In [45]:
# разметка из collection3
pd.read_csv('/content/drive/MyDrive/Collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


# NER из Spacy


In [46]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [47]:
#список токенов, частей речи и сущностей
for token in article:
    print(token.text, token.pos_, token.dep_)

" PUNCT punct
Яблоко NOUN nsubj
" PUNCT punct
выдвинуло VERB ROOT
Геннадия PROPN obj
Гудкова PROPN flat:name
в ADP case
губернаторы NOUN obl
Подмосковья PROPN nmod


 SPACE dep
Делегаты NOUN nsubj
конференции NOUN nmod
выдвигали VERB parataxis
также ADV advmod
кандидатуру NOUN obj
" PUNCT punct
яблочника NOUN nmod
" PUNCT punct
Кирилла PROPN appos
Янкова PROPN flat:name
, PUNCT punct
однако ADV advmod
он PRON nsubj
снял VERB conj
свою DET det
кандидатуру NOUN obj
. PUNCT punct
Таким DET det
образом NOUN parataxis
, PUNCT punct
фамилия NOUN nsubj
Гудкова PROPN nmod
была AUX cop
единственной ADJ ROOT
в ADP case
бюллетенях NOUN nmod
для ADP case
голосования NOUN nmod
. PUNCT punct


 SPACE dep
Конференция NOUN nsubj
подмосковного ADJ amod
отделение NOUN nmod
партии NOUN nmod
" PUNCT punct
Яблоко NOUN appos
" PUNCT punct
в ADP case
субботу NOUN obl
выдвинула VERB ROOT
экс NOUN obj
- NOUN obj
депутата NOUN obj
Госдумы PROPN nmod
Геннадия PROPN appos
Гудкова PROPN flat:name
кандидатом NOUN x

In [49]:
# разметка из collection3
pd.read_csv('/content/drive/MyDrive/Collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


# NER из slovnet

In [53]:
text = data_text.text[0]
navec = Navec.load('/navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('/slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

"Яблоко" выдвинуло Геннадия Гудкова в губернаторы Подмосковья
 ORG───            PER─────────────               LOC────────
Делегаты конференции выдвигали также кандидатуру "яблочника" Кирилла 
                                                             PER─────
Янкова, однако он снял свою кандидатуру. Таким образом, фамилия 
──────                                                          
Гудкова была единственной в бюллетенях для голосования.
PER────                                                
Конференция подмосковного отделение партии "Яблоко" в субботу 
                                            ORG───            
выдвинула экс-депутата Госдумы Геннадия Гудкова кандидатом на 
                       ORG──── PER─────────────               
сентябрьские выборы губернатора Московской области.
                                LOC─────────────── 
За его кандидатуру высказались 35, против — 4, один бюллетень был 
признан недействительным.
Делегаты конференции выдвигали также кандидат